In [1]:
from hello.hello3 import HelloApp
from hello import hello3
from hello import hello
from hello.pid.ag_pid import agpid
from hello.pid.ag_pid.agpid import *

In [2]:
def runagpid(r):
    try:
        r.doall()
    except Exception as e:
        import traceback
        traceback.print_exc()
    finally:
        return r
    
def test1(ip='71.189.82.196:88'):
    p = 0.1 * 36 / 59
    i = 0.01
    d = 0
    info=PIDInfo(p, i, d, b=0)
    app = hello3.open_hello(ip)
    tests = []
    for sp in (8, 10, 12, 14, 30, 55):
        t = AgOffToAutoTest(info, 0, sp, app)
        tests.append(t)
    for start_pv in (10, 12, 14):
        t1 = AgManToAutoTest(info, start_pv, start_pv-2, app)
        t2 = AgAutoToAutoTest(info, start_pv, start_pv-2, app)
        tests.extend((t1, t2))
    for start_pv in (8, 10, 12):
        t1 = AgManToAutoTest(info, start_pv, start_pv+2, app)
        t2 = AgAutoToAutoTest(info, start_pv, start_pv+2, app)
        tests.extend((t1, t2))
    r = PIDRunner(tests, wb_name='15L Mag Drive Agitation Test.xlsx')
    return runagpid(r)
    

In [3]:
r=test1()

02/16/17 15:42:15 INFO <runall>: Running test 1 of 18: P:0.061 I:0.010 D:0.0000 SP:8
02/16/17 15:42:15 INFO <run>: Beginning test run for OffToAuto SP:8.0
02/16/17 15:42:15 INFO <setup_settings>: Setting PID Settings
02/16/17 15:42:19 INFO <setup_test>: Setting Agitation to OFF and waiting 30 seconds
02/16/17 15:42:19 INFO <off>: Setting Agitation to Off
02/16/17 15:42:49 INFO <start_test>: Starting Test
02/16/17 15:42:49 INFO <auto>: Setting Agitation to 8RPM
02/16/17 15:42:49 INFO <wait_for_settle>: Waiting for PV to settle
02/16/17 15:42:49 INFO <wait_for_settle>: Update: PV: 0.00 SP: 8.00 Settle Remaining: 120 Timeout: 179
02/16/17 15:43:04 INFO <wait_for_settle>: Update: PV: 0.00 SP: 8.00 Settle Remaining: 120 Timeout: 164
02/16/17 15:43:19 INFO <wait_for_settle>: Update: PV: 5.93 SP: 8.00 Settle Remaining: 120 Timeout: 149
02/16/17 15:43:34 INFO <wait_for_settle>: Update: PV: 7.55 SP: 8.00 Settle Remaining: 113 Timeout: 134
02/16/17 15:43:49 INFO <wait_for_settle>: Update: PV: 8.

In [ ]:
def test2(ip='71.189.82.196:88'):
    p = 0.1 * 36 / 59
    i = 0.01
    d = 0
    app = hello3.open_hello(ip)
    tests = []
    for start_pv in (10, 12, 14):
        t2 = AgAutoToAutoTest(p, i, d, start_pv-2, start_pv, app)
        tests.append(t2)
    for start_pv in (8, 10, 12):
        t1 = AgManToAutoTest(p, i, d, start_pv+2, start_pv, app)
        t2 = AgAutoToAutoTest(p, i, d, start_pv+2, start_pv, app)
        # t3 = AgManToManTest(p, i, d, start_pv+2, start_pv, app)
        tests.extend((t1, t2))
    r = PIDRunner(tests, wb_name='test.xlsx')
    return runagpid(r)

In [12]:
def chartbypid(self, ifpassed=False):
    """
    @param ifpassed: Only plot tests that passed
    @type ifpassed: Bool

    Group all tests with the same p,i,d settings into one chart. Ie,
    all setpoints for a given set of settings in one chart.
    """
    groups = OrderedDict()
    for t in self._tests:

        if ifpassed and not t.passed:
            continue

        key = t.get_chart_key()
        if key not in groups:
            groups[key] = self._new_chart()
        t.chartplot(groups[key])

    # loop again and format + move charts to new sheets
    for key, chart in groups.items():
        p, i, d, other = key
        name = "P%sI%sD%s %s" % (fmt_float(p,3), fmt_float(i,3), fmt_float(d,3), other)
        name2 = "P: %s I: %s D:%s %s" % (p, i, d, other)
        title = "PID Test: " + name2
        self.logger.info("Formatting Chart: %s", name2)
        try:
            FormatChart(chart, ChartTitle=title, xAxisTitle="Time (s)", yAxisTitle="PV", Legend=True)
            chart.Location(1, name)
            axes = chart.Axes(AxisGroup=2)  # xlPrimary
            axes(1).HasMajorGridlines = True
            axes(2).HasMajorGridlines = True
        except Exception:
            self.logger.error("Error Formatting chart", exc_info=1)

    self._chartmap = groups

In [13]:
from types import MethodType
r.chartbypid = MethodType(chartbypid, r)

In [14]:
    def fmt_float(f, decimals=-1):
        if not f:
            return "0"
        f = str(f)
        if "." not in f:
            return f
        else:
            f = f.strip("0")
            if f[-1] == ".":
                f = f[:-1]
            if f[0] == ".":
                f = "0" + f
            decimals += 1
            if decimals > 0:
                f = f[:f.find(".")+decimals]
        return f

In [15]:
r._init_xl()

In [16]:
from officelib.xllib import *
r.plotall()
r.chartbypid()

02/16/17 17:13:48 INFO <plotall>: Copying test data to 15L Mag Drive Agitation Test.xlsx
02/16/17 17:13:48 INFO <plotall>: Copying data for test 1 of 18
02/16/17 17:13:49 INFO <plotall>: Copying data for test 2 of 18
02/16/17 17:13:49 INFO <plotall>: Copying data for test 3 of 18
02/16/17 17:13:49 INFO <plotall>: Copying data for test 4 of 18
02/16/17 17:13:49 INFO <plotall>: Copying data for test 5 of 18
02/16/17 17:13:49 INFO <plotall>: Copying data for test 6 of 18
02/16/17 17:13:49 INFO <plotall>: Copying data for test 7 of 18
02/16/17 17:13:49 INFO <plotall>: Copying data for test 8 of 18
02/16/17 17:13:49 INFO <plotall>: Copying data for test 9 of 18
02/16/17 17:13:49 INFO <plotall>: Copying data for test 10 of 18
02/16/17 17:13:49 INFO <plotall>: Copying data for test 11 of 18
02/16/17 17:13:49 INFO <plotall>: Copying data for test 12 of 18
02/16/17 17:13:49 INFO <plotall>: Copying data for test 13 of 18
02/16/17 17:13:49 INFO <plotall>: Copying data for test 14 of 18
02/16/17 1